Version 1- miu is the slope

In [ ]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
import matplotlib as mb
from scipy import stats

In [6]:
mb.use('TkAgg')

# Function to load a CSV file into a DataFrame
def load_csv_to_dataframe(file_path):
    try:
        df = pd.read_csv(file_path)
        print(f"Data loaded successfully from {file_path}")
        return df
    except Exception as e:
        print(f"Error loading data from {file_path}: {e}")
        return None

# Function to extract metadata and data from a CSV file
def extract_data(file_path):
    # Example usage
    data = load_csv_to_dataframe(file_path)

    # Splitting the original data into two sections for each channel
    channel_1 = data.iloc[:, :5]  # First 6 columns for Channel 1
    channel_2 = data.iloc[:, 6:11]  # Last 6 columns for Channel 2

    # Renaming columns for better readability
    channel_1.columns = ['metadata_name', 'metadata_value', 'metadata_units', 'T', 'V']
    channel_2.columns = ['metadata_name', 'metadata_value', 'metadata_units', 'T', 'V']

    # Extracting metadata and data for each channel
    channel_1_metadata = channel_1[['metadata_name', 'metadata_value', 'metadata_units']].transpose()
    channel_1_metadata.columns = channel_1_metadata.iloc[0]
    channel_1_metadata = channel_1_metadata[1:]

    channel_2_metadata = channel_2[['metadata_name', 'metadata_value', 'metadata_units']].transpose()
    channel_2_metadata.columns = channel_2_metadata.iloc[0]
    channel_2_metadata = channel_2_metadata[1:]

    channel_1 = channel_1[['T', 'V']]
    channel_2 = channel_2[['T', 'V']]

    return channel_1, channel_2

def find_simultaneous_max(ch1, ch2):
    # Find the index where ch1 is at its maximum
    ch1_max_idx = ch1['V'].idxmax()
    # Get the corresponding values from both channels
    return ch1.loc[ch1_max_idx, 'V'], ch2.loc[ch1_max_idx, 'V']

def create_separate_max_field_plots_with_regression():
    num_of_materials = np.arange(1, 5)
    resistance = [0.25, 0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    
    # Create a figure with 2x2 subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 15))
    axes = axes.ravel()  # Flatten axes array for easier indexing
    
    # Create a plot for each material
    for idx, material in enumerate(num_of_materials):
        max_ch1 = []
        max_ch2 = []
        
        # Collect data for each resistance value
        for res in resistance:
            file_path = f"data/2.3_material{material}_{res}kohm.csv"
            ch1_voltage, ch2_voltage = extract_data(file_path)
            
            # Find simultaneous maximum points
            ch1_max, ch2_max = find_simultaneous_max(ch1_voltage, ch2_voltage)
            max_ch1.append(ch1_max)
            max_ch2.append(ch2_max)
        
        # Convert to numpy arrays for regression
        max_ch1 = np.array(max_ch1)
        max_ch2 = np.array(max_ch2)
        
        # Perform linear regression
        slope, intercept, r_value, p_value, std_err = stats.linregress(max_ch1, max_ch2)
        
        # Create regression line
        line_x = np.linspace(min(max_ch1), max(max_ch1), 100)
        line_y = slope * line_x + intercept
        
        # Plot scatter points
        scatter = axes[idx].scatter(max_ch1, max_ch2, c=resistance, cmap='viridis', s=50)
        
        # Plot regression line
        axes[idx].plot(line_x, line_y, 'r--', 
                      label=f'Fit: y = {slope:.3f}x + {intercept:.3f}\nR² = {r_value**2:.3f}')
        
        axes[idx].set_xlabel("H (V)")
        axes[idx].set_ylabel("$\Phi_B$ (V)")
        axes[idx].set_title(f"Material {material} Maximum Points")
        axes[idx].grid(True, which='both', linestyle='--', linewidth=0.5)
        axes[idx].axhline(0, color='black', linewidth=1.5)
        axes[idx].axvline(0, color='black', linewidth=1.5)
        axes[idx].legend()
        
        # Add colorbar
        cbar = plt.colorbar(scatter, ax=axes[idx])
        cbar.set_label('Resistance (kΩ)')
    
    plt.tight_layout()
    plt.show()

# Call the function
create_separate_max_field_plots_with_regression()
'''''
def create_list_of_max_field_values():
    num_of_materials = np.arange(1, 5)
    fig = plt.figure(figsize=(10, 6))
    ax1 = fig.add_subplot(111)
    for material in num_of_materials:
        resistance= [0.25, 0.5, 1 , 2, 3 , 4 ,5 ,6, 7 ,8 ,9, 10 ,11]
        for res in resistance:
            file_path = f"../data/2.3_material{material}_{res}kohm.csv"
            ch1_voltage, ch2_voltage = extract_data(file_path)
            ax1.scatter(np.max(ch1_voltage), np.max(ch2_voltage), label=f"Material {material}", s=10)
    
    plt.xlabel("H (V)")
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.axhline(0, color='black', linewidth=1.5)  # Bold horizontal line at y=0
    plt.axvline(0, color='black', linewidth=1.5)  # Bold vertical line at x=0
    plt.ylabel("$\Phi_B$ (V)")
    plt.title("Hysteresis Loops of Different Materials")
    plt.legend()
    plt.show()

def create_list_of_all_loops():
    num_of_materials = np.arange(1, 5)
    fig = plt.figure(figsize=(10, 6))
    ax1 = fig.add_subplot(111)
    for material in num_of_materials:
        file_path = f"../data/2.3_material{material}.csv"
        ch1_voltage, ch2_voltage = extract_data(file_path)
        ax1.scatter(ch1_voltage, ch2_voltage, label=f"Material {material}", s=10)
'''''
    

#create_list_of_all_loops()


#create_list_of_max_field_values()

Data loaded successfully from data/2.3_material1_0.25kohm.csv
Data loaded successfully from data/2.3_material1_0.5kohm.csv
Data loaded successfully from data/2.3_material1_1kohm.csv
Data loaded successfully from data/2.3_material1_2kohm.csv
Data loaded successfully from data/2.3_material1_3kohm.csv
Data loaded successfully from data/2.3_material1_4kohm.csv
Data loaded successfully from data/2.3_material1_5kohm.csv
Data loaded successfully from data/2.3_material1_6kohm.csv
Data loaded successfully from data/2.3_material1_7kohm.csv
Data loaded successfully from data/2.3_material1_8kohm.csv
Data loaded successfully from data/2.3_material1_9kohm.csv
Data loaded successfully from data/2.3_material1_10kohm.csv
Data loaded successfully from data/2.3_material1_11kohm.csv
Data loaded successfully from data/2.3_material2_0.25kohm.csv
Data loaded successfully from data/2.3_material2_0.5kohm.csv
Data loaded successfully from data/2.3_material2_1kohm.csv
Data loaded successfully from data/2.3_mater

KeyboardInterrupt: 

Version 2- devided to be only miu

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mb
from scipy import stats

mb.use('TkAgg')

# Function to load a CSV file into a DataFrame
def load_csv_to_dataframe(file_path):
    try:
        df = pd.read_csv(file_path)
        print(f"Data loaded successfully from {file_path}")
        return df
    except Exception as e:
        print(f"Error loading data from {file_path}: {e}")
        return None

# Function to extract metadata and data from a CSV file
def extract_data(file_path):
    data = load_csv_to_dataframe(file_path)
    
    # Splitting the original data into two sections for each channel
    channel_1 = data.iloc[:, :5]  # First 5 columns for Channel 1
    channel_2 = data.iloc[:, 6:11]  # Next 5 columns for Channel 2

    # Renaming columns for better readability
    channel_1.columns = ['metadata_name', 'metadata_value', 'metadata_units', 'T', 'V']
    channel_2.columns = ['metadata_name', 'metadata_value', 'metadata_units', 'T', 'V']

    # Extract the required data for each channel
    channel_1 = channel_1[['T', 'V']]
    channel_2 = channel_2[['T', 'V']]
    
    return channel_1, channel_2

# Function to calculate the H-field and B-field
def calculate_h_and_b(ch1, ch2, scaling_H=1.0, scaling_B=1.0):
    H_field = ch1['V'] * scaling_H  # H-field proportional to voltage
    B_field = np.cumsum(ch2['V']) * scaling_B  # B-field as cumulative sum (integral)
    return H_field, B_field

# Function to find simultaneous maximum points
def find_simultaneous_max(H_field, B_field):
    # Combine H and B into a DataFrame for synchronization
    combined = pd.DataFrame({'H': H_field, 'B': B_field})
    max_idx = combined['H'].idxmax()  # Find index of max H (assume same time index for max H and B)
    return combined.loc[max_idx, 'H'], combined.loc[max_idx, 'B']

# Function to generate permeability from simultaneous maximum points
def generate_permeability_from_max_points():
    num_of_materials = np.arange(1, 5)
    resistance = [0.25, 0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 15))
    axes = axes.ravel()  # Flatten axes array for easier indexing
    
    for idx, material in enumerate(num_of_materials):
        max_H_values = []
        max_B_values = []
        resistances = []

        for res in resistance:
            file_path = f"data/2.3_material{material}_{res}kohm.csv"
            ch1, ch2 = extract_data(file_path)
            
            # Calculate H-field and B-field
            H_field, B_field = calculate_h_and_b(ch1, ch2, scaling_H=1.0, scaling_B=0.1)
            
            # Find simultaneous maximum points
            max_H, max_B = find_simultaneous_max(H_field, B_field)
            max_H_values.append(max_H)
            max_B_values.append(max_B)
            resistances.append(res)
        
        # Calculate permeability (μ = B / H)
        permeability = np.array(max_B_values) / np.array(max_H_values)
        
        # Plot permeability vs resistance
        axes[idx].plot(resistances, permeability, 'o-', label="Permeability")
        axes[idx].set_xlabel("Resistance (kΩ)")
        axes[idx].set_ylabel("Permeability (μ)")
        axes[idx].set_title(f"Material {material}")
        axes[idx].grid(True, which='both', linestyle='--', linewidth=0.5)
        axes[idx].legend()
    
    plt.tight_layout()
    plt.show()

# Call the function
generate_permeability_from_max_points()

